In [ ]:
!pip install -q PyDrive

In [ ]:
import os
import cv2
import keras
import shutil
import zipfile
import numpy as np
import keras.callbacks
import tensorflow as tf
import matplotlib.pyplot as plt

from google.colab import drive

data_dir = '/tmp/data'
train_dir = '/tmp/traindata'
validation_dir = '/tmp/validationdata'
images_dir, masks_dir = 'images', 'masks'

print(tf.__version__)

In [ ]:
drive.mount('/content/gdrive')

In [ ]:
try:
  shutil.rmtree(data_dir)
except:
  pass

zip_ref = zipfile.ZipFile('/content/gdrive/My Drive/data.zip', 'r')
zip_ref.extractall(data_dir)
zip_ref.close()

In [ ]:
try:
  shutil.rmtree(train_dir)
except:
  pass
os.mkdir(train_dir)

try:
  shutil.rmtree(validation_dir)
except:
  pass
os.mkdir(validation_dir)

split = .95
file_names = os.listdir(os.path.join(data_dir, masks_dir))
split_index = int(split * len(file_names))

try:
  shutil.rmtree(os.path.join(train_dir, images_dir))
except:
  pass
os.mkdir(os.path.join(train_dir, images_dir))

try:
  shutil.rmtree(os.path.join(train_dir, masks_dir))
except:
  pass
os.mkdir(os.path.join(train_dir, masks_dir))

train_names = file_names[:split_index]
np.random.shuffle(train_names)

for file_name in train_names:
  shutil.copyfile(os.path.join(data_dir, images_dir, file_name),
                  os.path.join(train_dir, images_dir, file_name))
  shutil.copyfile(os.path.join(data_dir, masks_dir, file_name),
                  os.path.join(train_dir, masks_dir, file_name))

try:
  shutil.rmtree(os.path.join(validation_dir, images_dir))
except:
  pass

os.mkdir(os.path.join(validation_dir, images_dir))

try:
  shutil.rmtree(os.path.join(validation_dir, masks_dir))
except:
  pass

os.mkdir(os.path.join(validation_dir, masks_dir))

validation_names = file_names[split_index:]
np.random.shuffle(validation_names)
for file_name in validation_names:
  shutil.copyfile(os.path.join(data_dir, images_dir, file_name),
                  os.path.join(validation_dir, images_dir, file_name))
  shutil.copyfile(os.path.join(data_dir, masks_dir, file_name),
                  os.path.join(validation_dir, masks_dir, file_name))

In [ ]:
image_shape = (1280, 768)

def data_generator(data_path, seed=1): 
  image_data_generator = keras.preprocessing.image.ImageDataGenerator()

  image_generator = image_data_generator.flow_from_directory(
        data_path, classes=[images_dir], class_mode=None,
        color_mode='grayscale', target_size=image_shape, batch_size=1,
        seed=seed)
  
  mask_generator = image_data_generator.flow_from_directory(
      data_path, classes=[masks_dir], class_mode=None,
      color_mode='grayscale', target_size=image_shape, batch_size=1,
      seed=seed)
  
  for img, mask in zip(image_generator, mask_generator):
    img /= 255
    mask /= 255

    yield img, mask

In [ ]:
train_generator_instance = data_generator(train_dir)

for _ in range(1):
  image, mask = next(train_generator_instance)
  plt.imshow(np.hstack((np.squeeze(image[0] * 255).astype(np.uint8),
                        np.invert(np.squeeze(mask[0] * 255).astype(np.uint8)))),
             cmap='gray')
  plt.show()

validation_generator_instance = data_generator(validation_dir)

for _ in range(1):
  image, mask = next(validation_generator_instance)
  plt.imshow(np.hstack((np.squeeze(image[0] * 255).astype(np.uint8),
                        np.invert(np.squeeze(mask[0] * 255).astype(np.uint8)))),
             cmap='gray')
  plt.show()

In [ ]:
def unet():
    inputs = keras.layers.Input((image_shape[0], image_shape[1], 1))

    conv1 = keras.layers.Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
    conv1 = keras.layers.Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
    pool1 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = keras.layers.Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
    conv2 = keras.layers.Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
    pool2 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = keras.layers.Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
    conv3 = keras.layers.Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
    pool3 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = keras.layers.Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool3)
    conv4 = keras.layers.Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv4)
    drop4 = keras.layers.Dropout(0.5)(conv4)
    pool4 = keras.layers.MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = keras.layers.Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool4)
    conv5 = keras.layers.Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv5)
    drop5 = keras.layers.Dropout(0.5)(conv5)

    up6 = keras.layers.Conv2D(512, 2, activation='relu', padding='same', kernel_initializer='he_normal')(keras.layers.UpSampling2D(size = (2,2))(drop5))
    merge6 = keras.layers.concatenate([drop4, up6], axis=3)
    conv6 = keras.layers.Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge6)
    conv6 = keras.layers.Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv6)

    up7 = keras.layers.Conv2D(256, 2, activation='relu', padding='same', kernel_initializer='he_normal')(keras.layers.UpSampling2D(size = (2,2))(conv6))
    merge7 = keras.layers.concatenate([conv3, up7], axis = 3)
    conv7 = keras.layers.Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge7)
    conv7 = keras.layers.Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv7)

    up8 = keras.layers.Conv2D(128, 2, activation='relu', padding='same', kernel_initializer='he_normal')(keras.layers.UpSampling2D(size = (2,2))(conv7))
    merge8 = keras.layers.concatenate([conv2, up8], axis = 3)
    conv8 = keras.layers.Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge8)
    conv8 = keras.layers.Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv8)

    up9 = keras.layers.Conv2D(64, 2, activation='relu', padding='same', kernel_initializer='he_normal')(keras.layers.UpSampling2D(size = (2,2))(conv8))
    merge9 = keras.layers.concatenate([conv1, up9], axis=3)

    conv9 = keras.layers.Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge9)
    conv9 = keras.layers.Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
    conv9 = keras.layers.Conv2D(2, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
    conv10 = keras.layers.Conv2D(1, 1, activation='sigmoid')(conv9)

    model = keras.models.Model(input=inputs, output=conv10)
    model.compile(optimizer=keras.optimizers.Adam(lr=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
model = unet()

In [ ]:
weights_checkpoint_dir = '/content/gdrive/My Drive/unet_snapshot/weights.h5'

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',
                                               min_delta=0,
                                               patience=20,
                                               verbose=0,
                                               mode='auto')
model_checkpoint = keras.callbacks.ModelCheckpoint(weights_checkpoint_dir,
                                                   monitor='val_loss',
                                                   save_best_only=True)

callbacks = [model_checkpoint, early_stopping]

In [ ]:
history = model.fit_generator(train_generator_instance,
                              steps_per_epoch=3536,
                              epochs=250,
                              verbose=1,
                              validation_data=validation_generator_instance,
                              validation_steps=187,
                              shuffle=True,
                              callbacks=callbacks)

In [ ]:
model.load_weights(weights_checkpoint_dir)

In [ ]:
image, mask = next(validation_generator_instance)

output = model.predict([image], verbose=1)
result = np.hstack((np.invert(np.squeeze(mask * 255).astype(np.uint8)),
                    np.squeeze(image * 255).astype(np.uint8),
                    np.invert(np.squeeze(output * 255).astype(np.uint8))))

plt.imshow(result, cmap='gray')
plt.show()

In [ ]:
cv2.imwrite('/content/gdrive/My Drive/result.png', result)